<a href="https://colab.research.google.com/github/BigyanBhatta/KerasTuner/blob/main/KerasTuner_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from mlxtend.plotting import plot_decision_regions


import kerastuner as kt


In [11]:
df = pd.read_csv('/content/diabetes.csv')

In [12]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [14]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [15]:
X = df.iloc[:, :-1].values

In [16]:
y = df.iloc[: , -1].values

In [17]:
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)


In [19]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [20]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 32, verbose = False)

In [21]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation = 'relu', input_dim = 8))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta']), loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model



In [22]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5,
)

In [23]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 06s


In [24]:
tuner.get_best_hyperparameters()[0].values


{'optimizer': 'sgd'}

In [25]:
model = tuner.get_best_models(num_models= 1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
model.fit(X_train, y_train, epochs = 100, batch_size = 32, initial_epoch= 5, validation_data = (X_test, y_test), verbose = False)


In [28]:
def build_model(hp):
  model = Sequential()
  unit = hp.Int('units',min_value = 8,max_value =  128, step = 8)
  model.add(Dense(unit, activation = 'relu', input_dim = 8))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model


In [41]:
tuner = kt.RandomSearch(
    hypermodel = build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    directory = 'mydir'
)

In [42]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 08s


In [43]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [44]:
model = tuner.get_best_models(num_models = 1)[0]

In [47]:
model.fit(X_train, y_train, epochs = 100, initial_epoch = 5, validation_data = (X_test, y_test), verbose = False)

In [49]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(128, activation = 'relu', input_dim = 8))

  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    model.add(Dense(128, activation = 'relu'))

  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model


In [50]:
tuner = kt.RandomSearch(hypermodel = build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydir',
                        project_name = 'NumLayers')

In [51]:
tuner.search(X_train,y_train, validation_data = (X_test, y_test), epochs = 5)

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 22s


In [52]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [53]:
model = tuner.get_best_models(num_models = 1)[0]

In [55]:
model.fit(X_train, y_train, epochs = 100, initial_epoch = 5, validation_data = (X_test, y_test), verbose = False)

In [74]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value =1, max_value = 10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value= 8 , max_value = 128, step = 8), activation = hp.Choice('activation' + str(i), values= ['relu', 'elu', 'tanh']), input_dim = 8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))

    else:
      model.add(Dense(hp.Int('units' + str(i), min_value= 8 , max_value = 128, step = 8), activation = hp.Choice('activation' + str(i), values = ['relu', 'elu', 'tanh'])))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))

    counter += 1

  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer = hp.Choice('optimizer', values = ['rmsprop', 'adam', 'adadelta']), loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model






In [75]:
tuner = kt.RandomSearch(hypermodel = build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydir',
                        project_name = 'final')

Reloading Tuner from mydir/final/tuner0.json


In [76]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

In [80]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 120,
 'activation0': 'elu',
 'optimizer': 'rmsprop',
 'units1': 128,
 'activation1': 'elu',
 'units2': 96,
 'activation2': 'elu',
 'units3': 40,
 'activation3': 'relu',
 'units4': 104,
 'activation4': 'elu',
 'units5': 80,
 'activation5': 'relu',
 'units6': 72,
 'activation6': 'tanh',
 'units7': 56,
 'activation7': 'elu',
 'units8': 112,
 'activation8': 'tanh',
 'dropout0': 0.1,
 'dropout1': 0.1,
 'dropout2': 0.1}

In [81]:
model = tuner.get_best_models(num_models = 1)[0]


In [82]:
model.fit(X_train, y_train, epochs = 200, initial_epoch = 5, validation_data = (X_test, y_test))

Epoch 6/200
20/20 [==============================] - 1s 14ms/step - loss: 0.4743 - accuracy: 0.7752 - val_loss: 0.4494 - val_accuracy: 0.7792
Epoch 7/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4681 - accuracy: 0.7622 - val_loss: 0.4476 - val_accuracy: 0.8052
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4685 - accuracy: 0.7671 - val_loss: 0.4503 - val_accuracy: 0.8052
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4700 - accuracy: 0.7785 - val_loss: 0.4754 - val_accuracy: 0.8052
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4644 - accuracy: 0.7687 - val_loss: 0.4668 - val_accuracy: 0.8117
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4638 - accuracy: 0.7785 - val_loss: 0.4967 - val_accuracy: 0.7857
Epoch 12/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4675 - accuracy: 0.7606 - val_loss: 0.4634 - val_accuracy: 0.8052
Epoch 13/